In [1]:
import pointblank as pb
import polars as pl

## Basic

In [2]:
#  ❗ sudo code
# (
#     pb.Validate(data=)
#     .validation_steps()
#     .interrogate()
# )

In [3]:
small_table = pb.load_dataset(dataset="small_table", tbl_type="polars")
print(small_table.columns)
print(small_table.shape)

['date_time', 'date', 'a', 'b', 'c', 'd', 'e', 'f']
(13, 8)


In [4]:
print(small_table.select(pl.col("a").max()))

shape: (1, 1)
┌─────┐
│ a   │
│ --- │
│ i64 │
╞═════╡
│ 8   │
└─────┘


In [5]:
print(small_table.select(pl.col("f").unique()))

shape: (3, 1)
┌──────┐
│ f    │
│ ---  │
│ str  │
╞══════╡
│ low  │
│ mid  │
│ high │
└──────┘


### Example1

In [6]:
pb.Validate(small_table).col_vals_le("a", 8).interrogate()

Validate(data=shape: (13, 8)
┌─────────────────────┬────────────┬─────┬───────────┬──────┬─────────┬───────┬──────┐
│ date_time           ┆ date       ┆ a   ┆ b         ┆ c    ┆ d       ┆ e     ┆ f    │
│ ---                 ┆ ---        ┆ --- ┆ ---       ┆ ---  ┆ ---     ┆ ---   ┆ ---  │
│ datetime[μs]        ┆ date       ┆ i64 ┆ str       ┆ i64  ┆ f64     ┆ bool  ┆ str  │
╞═════════════════════╪════════════╪═════╪═══════════╪══════╪═════════╪═══════╪══════╡
│ 2016-01-04 11:00:00 ┆ 2016-01-04 ┆ 2   ┆ 1-bcd-345 ┆ 3    ┆ 3423.29 ┆ true  ┆ high │
│ 2016-01-04 00:32:00 ┆ 2016-01-04 ┆ 3   ┆ 5-egh-163 ┆ 8    ┆ 9999.99 ┆ true  ┆ low  │
│ 2016-01-05 13:32:00 ┆ 2016-01-05 ┆ 6   ┆ 8-kdg-938 ┆ 3    ┆ 2343.23 ┆ true  ┆ high │
│ 2016-01-06 17:23:00 ┆ 2016-01-06 ┆ 2   ┆ 5-jdo-903 ┆ null ┆ 3892.4  ┆ false ┆ mid  │
│ 2016-01-09 12:36:00 ┆ 2016-01-09 ┆ 8   ┆ 3-ldm-038 ┆ 7    ┆ 283.94  ┆ true  ┆ low  │
│ …                   ┆ …          ┆ …   ┆ …         ┆ …    ┆ …       ┆ …     ┆ …    │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-26 20:07:00 ┆ 2016-01-26 ┆ 4   ┆ 2-dmx-010 ┆ 7    ┆ 833.98  ┆ true  ┆ low  │
│ 2016-01-28 02:51:00 ┆ 2016-01-28 ┆ 2   ┆ 7-dmx-010 ┆ 8    ┆ 108.34  ┆ false ┆ low  │
│ 2016-01-30 11:23:00 ┆ 2016-01-30 ┆ 1   ┆ 3-dka-303 ┆ null ┆ 2230.09 ┆ true  ┆ high │
└─────────────────────┴────────────┴─────┴───────────┴──────┴─────────┴───────┴──────┘, tbl_name=None, label=None, thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, final_actions=None, brief=None, lang='en', locale='en')

### Example2

In [7]:
def send_alert():
    summary = pb.get_validation_summary()
    if summary["highest_severity"] == "critical":
        print(
            f"ALERT: Critical validation failures found in {summary['tbl_name']}"
        )


(
    pb.Validate(
        data=small_table,
        tbl_name="Small Table",
        label="Polars熊霸天下[Day28]",
        thresholds=pb.Thresholds(warning=0.1, error=0.2, critical=0.3),
        actions=pb.Actions(error="ERROR DETECTED: {type}"),
        final_actions=pb.FinalActions(send_alert),
        brief=True,
        lang="zh-Hant",
    )
    .col_vals_le("a", 6)
    .col_vals_in_set(
        "f",
        ["mid", "high"],
        actions=pb.Actions(warning="WARNING DETECTED: {type}"),
    )
    .interrogate()
)

WARNING DETECTED: col_vals_in_set
ALERT: Critical validation failures found in Small Table


Validate(data=shape: (13, 8)
┌─────────────────────┬────────────┬─────┬───────────┬──────┬─────────┬───────┬──────┐
│ date_time           ┆ date       ┆ a   ┆ b         ┆ c    ┆ d       ┆ e     ┆ f    │
│ ---                 ┆ ---        ┆ --- ┆ ---       ┆ ---  ┆ ---     ┆ ---   ┆ ---  │
│ datetime[μs]        ┆ date       ┆ i64 ┆ str       ┆ i64  ┆ f64     ┆ bool  ┆ str  │
╞═════════════════════╪════════════╪═════╪═══════════╪══════╪═════════╪═══════╪══════╡
│ 2016-01-04 11:00:00 ┆ 2016-01-04 ┆ 2   ┆ 1-bcd-345 ┆ 3    ┆ 3423.29 ┆ true  ┆ high │
│ 2016-01-04 00:32:00 ┆ 2016-01-04 ┆ 3   ┆ 5-egh-163 ┆ 8    ┆ 9999.99 ┆ true  ┆ low  │
│ 2016-01-05 13:32:00 ┆ 2016-01-05 ┆ 6   ┆ 8-kdg-938 ┆ 3    ┆ 2343.23 ┆ true  ┆ high │
│ 2016-01-06 17:23:00 ┆ 2016-01-06 ┆ 2   ┆ 5-jdo-903 ┆ null ┆ 3892.4  ┆ false ┆ mid  │
│ 2016-01-09 12:36:00 ┆ 2016-01-09 ┆ 8   ┆ 3-ldm-038 ┆ 7    ┆ 283.94  ┆ true  ┆ low  │
│ …                   ┆ …          ┆ …   ┆ …         ┆ …    ┆ …       ┆ …     ┆ …    │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-26 20:07:00 ┆ 2016-01-26 ┆ 4   ┆ 2-dmx-010 ┆ 7    ┆ 833.98  ┆ true  ┆ low  │
│ 2016-01-28 02:51:00 ┆ 2016-01-28 ┆ 2   ┆ 7-dmx-010 ┆ 8    ┆ 108.34  ┆ false ┆ low  │
│ 2016-01-30 11:23:00 ┆ 2016-01-30 ┆ 1   ┆ 3-dka-303 ┆ null ┆ 2230.09 ┆ true  ┆ high │
└─────────────────────┴────────────┴─────┴───────────┴──────┴─────────┴───────┴──────┘, tbl_name='Small Table', label='Polars熊霸天下[Day28]', thresholds=Thresholds(warning=0.1, error=0.2, critical=0.3), actions=Actions(warning=None, error=['ERROR DETECTED: {type}'], critical=None), final_actions=FinalActions(send_alert), brief='{auto}', lang='zh-Hant', locale='zh-Hant')

## yaml

### From Example1

In [8]:
yaml_file = "small_table.yaml"
pb.yaml_interrogate(yaml_file)

Validate(data=shape: (13, 8)
┌─────────────────────┬────────────┬─────┬───────────┬──────┬─────────┬───────┬──────┐
│ date_time           ┆ date       ┆ a   ┆ b         ┆ c    ┆ d       ┆ e     ┆ f    │
│ ---                 ┆ ---        ┆ --- ┆ ---       ┆ ---  ┆ ---     ┆ ---   ┆ ---  │
│ datetime[μs]        ┆ date       ┆ i64 ┆ str       ┆ i64  ┆ f64     ┆ bool  ┆ str  │
╞═════════════════════╪════════════╪═════╪═══════════╪══════╪═════════╪═══════╪══════╡
│ 2016-01-04 11:00:00 ┆ 2016-01-04 ┆ 2   ┆ 1-bcd-345 ┆ 3    ┆ 3423.29 ┆ true  ┆ high │
│ 2016-01-04 00:32:00 ┆ 2016-01-04 ┆ 3   ┆ 5-egh-163 ┆ 8    ┆ 9999.99 ┆ true  ┆ low  │
│ 2016-01-05 13:32:00 ┆ 2016-01-05 ┆ 6   ┆ 8-kdg-938 ┆ 3    ┆ 2343.23 ┆ true  ┆ high │
│ 2016-01-06 17:23:00 ┆ 2016-01-06 ┆ 2   ┆ 5-jdo-903 ┆ null ┆ 3892.4  ┆ false ┆ mid  │
│ 2016-01-09 12:36:00 ┆ 2016-01-09 ┆ 8   ┆ 3-ldm-038 ┆ 7    ┆ 283.94  ┆ true  ┆ low  │
│ …                   ┆ …          ┆ …   ┆ …         ┆ …    ┆ …       ┆ …     ┆ …    │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ 5-bce-642 ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 2016-01-26 20:07:00 ┆ 2016-01-26 ┆ 4   ┆ 2-dmx-010 ┆ 7    ┆ 833.98  ┆ true  ┆ low  │
│ 2016-01-28 02:51:00 ┆ 2016-01-28 ┆ 2   ┆ 7-dmx-010 ┆ 8    ┆ 108.34  ┆ false ┆ low  │
│ 2016-01-30 11:23:00 ┆ 2016-01-30 ┆ 1   ┆ 3-dka-303 ┆ null ┆ 2230.09 ┆ true  ┆ high │
└─────────────────────┴────────────┴─────┴───────────┴──────┴─────────┴───────┴──────┘, tbl_name=None, label=None, thresholds=Thresholds(warning=None, error=None, critical=None), actions=None, final_actions=None, brief=None, lang='en', locale='en')

In [9]:
print(pb.yaml_to_python(yaml_file))

```python
import pointblank as pb

(
    pb.Validate(data=pb.load_dataset("small_table", tbl_type="polars"))
    .col_vals_le(columns="a", value=8)
    .interrogate()
)
```


## Inspection 

In [10]:
pb.preview(small_table)

GT(_tbl_data=shape: (10, 9)
┌───────────┬─────────────────────┬────────────┬─────┬───┬──────┬─────────┬───────┬──────┐
│ _row_num_ ┆ date_time           ┆ date       ┆ a   ┆ … ┆ c    ┆ d       ┆ e     ┆ f    │
│ ---       ┆ ---                 ┆ ---        ┆ --- ┆   ┆ ---  ┆ ---     ┆ ---   ┆ ---  │
│ i64       ┆ datetime[μs]        ┆ date       ┆ i64 ┆   ┆ i64  ┆ f64     ┆ bool  ┆ str  │
╞═══════════╪═════════════════════╪════════════╪═════╪═══╪══════╪═════════╪═══════╪══════╡
│ 1         ┆ 2016-01-04 11:00:00 ┆ 2016-01-04 ┆ 2   ┆ … ┆ 3    ┆ 3423.29 ┆ true  ┆ high │
│ 2         ┆ 2016-01-04 00:32:00 ┆ 2016-01-04 ┆ 3   ┆ … ┆ 8    ┆ 9999.99 ┆ true  ┆ low  │
│ 3         ┆ 2016-01-05 13:32:00 ┆ 2016-01-05 ┆ 6   ┆ … ┆ 3    ┆ 2343.23 ┆ true  ┆ high │
│ 4         ┆ 2016-01-06 17:23:00 ┆ 2016-01-06 ┆ 2   ┆ … ┆ null ┆ 3892.4  ┆ false ┆ mid  │
│ 5         ┆ 2016-01-09 12:36:00 ┆ 2016-01-09 ┆ 8   ┆ … ┆ 7    ┆ 283.94  ┆ true  ┆ low  │
│ 9         ┆ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ … ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 10        ┆ 2016-01-20 04:30:00 ┆ 2016-01-20 ┆ 3   ┆ … ┆ 9    ┆ 837.93  ┆ false ┆ high │
│ 11        ┆ 2016-01-26 20:07:00 ┆ 2016-01-26 ┆ 4   ┆ … ┆ 7    ┆ 833.98  ┆ true  ┆ low  │
│ 12        ┆ 2016-01-28 02:51:00 ┆ 2016-01-28 ┆ 2   ┆ … ┆ 8    ┆ 108.34  ┆ false ┆ low  │
│ 13        ┆ 2016-01-30 11:23:00 ┆ 2016-01-30 ┆ 1   ┆ … ┆ null ┆ 2230.09 ┆ true  ┆ high │
└───────────┴─────────────────────┴────────────┴─────┴───┴──────┴─────────┴───────┴──────┘, _body=<great_tables._gt_data.Body object at 0x000001AEBC0B38E0>, _boxhead=Boxhead([ColInfo(var='_row_num_', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='right', column_width='25.6px'), ColInfo(var='date_time', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>date_time</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Datetime</em></div></div>"), column_align='right', column_width='158px'), ColInfo(var='date', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>date</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Date</em></div></div>"), column_align='right', column_width='88px'), ColInfo(var='a', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>a</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Int64</em></div></div>"), column_align='right', column_width='49px'), ColInfo(var='b', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>b</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>String</em></div></div>"), column_align='left', column_width='80px'), ColInfo(var='c', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>c</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Int64</em></div></div>"), column_align='right', column_width='49px'), ColInfo(var='d', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>d</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Fl

In [11]:
pb.preview(
    small_table,
    columns_subset=["date_time", "date", "a", "b", "c"],
    n_head=2,
    n_tail=2,
)

GT(_tbl_data=shape: (4, 6)
┌───────────┬─────────────────────┬────────────┬─────┬───────────┬──────┐
│ _row_num_ ┆ date_time           ┆ date       ┆ a   ┆ b         ┆ c    │
│ ---       ┆ ---                 ┆ ---        ┆ --- ┆ ---       ┆ ---  │
│ i64       ┆ datetime[μs]        ┆ date       ┆ i64 ┆ str       ┆ i64  │
╞═══════════╪═════════════════════╪════════════╪═════╪═══════════╪══════╡
│ 1         ┆ 2016-01-04 11:00:00 ┆ 2016-01-04 ┆ 2   ┆ 1-bcd-345 ┆ 3    │
│ 2         ┆ 2016-01-04 00:32:00 ┆ 2016-01-04 ┆ 3   ┆ 5-egh-163 ┆ 8    │
│ 12        ┆ 2016-01-28 02:51:00 ┆ 2016-01-28 ┆ 2   ┆ 7-dmx-010 ┆ 8    │
│ 13        ┆ 2016-01-30 11:23:00 ┆ 2016-01-30 ┆ 1   ┆ 3-dka-303 ┆ null │
└───────────┴─────────────────────┴────────────┴─────┴───────────┴──────┘, _body=<great_tables._gt_data.Body object at 0x000001AEBC116EE0>, _boxhead=Boxhead([ColInfo(var='_row_num_', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='right', column_width='25.6px'), ColInfo(var='date_time', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>date_time</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Datetime</em></div></div>"), column_align='right', column_width='173px'), ColInfo(var='date', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>date</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Date</em></div></div>"), column_align='right', column_width='103px'), ColInfo(var='a', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>a</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Int64</em></div></div>"), column_align='right', column_width='64px'), ColInfo(var='b', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>b</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>String</em></div></div>"), column_align='left', column_width='95px'), ColInfo(var='c', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>c</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Int64</em></div></div>"), column_align='right', column_width='64px')]), _stub=<great_tables._gt_data.Stub object at 0x000001AEBC0CCAD0>, _spanners=Spanners([]), _heading=Heading(title=Html(text='<div><div style="padding-top: 0; padding-bottom: 7px;"><span style=\'background-color: #0075FF; color: #FFFFFF; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; margin: 5px 10px 5px 0px; border: solid 1px #0075FF; font-weight: bold; padding: 2px 10px 2px 10px; font-size: 10px;\'>Polars</span><span style=\'background-color: #eecbff; color: #333333; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; margin: 5px 0px 5px 5px; font-weight: bold; border: solid 1px #eecbff; padding: 2px 15px 2px 15px; font-size: 10px;\'>Rows</span><span style=\'background-color: none; color: #333333; padding: 0.5em 0.5em; position: inherit; margin: 5px 0px 5px -4px; font-weight: bold; border: solid 1px #eecbff; padding: 2px 15px 2px 15px; font-size: 10px;\'>13</span><span style=\'background-color: #BDE7B4; color: #333333; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; m

In [12]:
pb.col_summary_tbl(small_table)

GT(_tbl_data=shape: (8, 14)
┌────────────┬─────────┬───────────┬────────────┬───┬─────────┬────────────┬──────────┬────────────┐
│ mean       ┆ q_3     ┆ n_missing ┆ min        ┆ … ┆ p05     ┆ std        ┆ p95      ┆ colname    │
│ ---        ┆ ---     ┆ ---       ┆ ---        ┆   ┆ ---     ┆ ---        ┆ ---      ┆ ---        │
│ f64        ┆ f64     ┆ str       ┆ str        ┆   ┆ f64     ┆ f64        ┆ f64      ┆ str        │
╞════════════╪═════════╪═══════════╪════════════╪═══╪═════════╪════════════╪══════════╪════════════╡
│ null       ┆ null    ┆ 0<br>0    ┆ 2016<br>01 ┆ … ┆ null    ┆ null       ┆ null     ┆ <div style │
│            ┆         ┆           ┆ <br>04     ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆ 00:32:00   ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ null       ┆ null    ┆ 0<br>0    ┆ 2016<br>01 ┆ … ┆ null    ┆ null       ┆ null     ┆ <div style │
│            ┆         ┆           ┆ <br>04     ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ 3.769231   ┆ 4.0     ┆ 0<br>0    ┆ 1          ┆ … ┆ 1.06    ┆ 2.087816   ┆ 7.4      ┆ <div style │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ 9.0        ┆ 9.0     ┆ 0<br>0    ┆ 9          ┆ … ┆ 9.0     ┆ 0.0        ┆ 9.0      ┆ <div style │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ 5.727273   ┆ 8.0     ┆ 2<br>0.15 ┆ 2          ┆ … ┆ 2.05    ┆ 2.723634   ┆ 9.0      ┆ <div style │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ 2304.70230 ┆ 3291.03 ┆ 0<br>0    ┆ 108.34     ┆ … ┆ 118.876 ┆ 2631.36247 ┆ 6335.436 ┆ <div style │
│ 8          ┆         ┆           ┆            ┆   ┆         ┆ 2          ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ null       ┆ null    ┆ 0<br>0    ┆ null       ┆ … ┆ null    ┆ null       ┆ null     ┆ <div style │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
│ 3.461538   ┆ 4.0     ┆ 0<br>0    ┆ 3          ┆ … ┆ 3.0     ┆ 0.518875   ┆ 4.0      ┆ <div style │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ ='font-siz │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ e: 13px;   │
│            ┆         ┆           ┆            ┆   ┆         ┆            ┆          ┆ w…         │
└────────────┴─────────┴───────────┴────────────┴───┴─────────┴────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000001AEBC0E6760>, _boxhead=Boxhead([ColInfo(var='icon', type=<ColInfoTypeEnum.default:

In [13]:
pb.missing_vals_tbl(small_table)

GT(_tbl_data=shape: (8, 11)
┌───────────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬──────┐
│ columns   ┆ 1   ┆ 2   ┆ 3   ┆ … ┆ 7   ┆ 8   ┆ 9   ┆ 10   │
│ ---       ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ ---  │
│ str       ┆ f64 ┆ f64 ┆ f64 ┆   ┆ f64 ┆ f64 ┆ f64 ┆ f64  │
╞═══════════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪══════╡
│ date_time ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ date      ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ a         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ b         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ c         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 25.0 │
│ d         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ e         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ f         ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0.0  │
└───────────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴──────┘, _body=<great_tables._gt_data.Body object at 0x000001AEBC0EF1B0>, _boxhead=Boxhead([ColInfo(var='columns', type=<ColInfoTypeEnum.default: 1>, column_label='Column', column_align='left', column_width='200px'), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width='30px'), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width='30px'), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width='30px'), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width='30px'), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='center', column_width='30px'), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='center', column_width='30px'), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='center', column_width='30px'), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='center', column_width='30px'), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='center', column_width='30px'), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='center', column_width='30px')]), _stub=<great_tables._gt_data.Stub object at 0x000001AEBC01F9B0>, _spanners=Spanners([SpannerInfo(spanner_id='Row Sector', spanner_level=0, spanner_label='Row Sector', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], built=None)]), _heading=Heading(title=Html(text="Missing Values&nbsp;&nbsp;&nbsp;<span style='font-size: 14px; text-transform: uppercase; color: #333333'>2 in total</span>"), subtitle=Html(text='<div><div style="padding-top: 0; padding-bottom: 7px;"><span style=\'background-color: #0075FF; color: #FFFFFF; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; margin: 5px 10px 5px 0px; border: solid 1px #0075FF; font-weight: bold; padding: 2px 10px 2px 10px; font-size: 10px;\'>Polars</span><span style=\'background-color: #eecbff; color: #333333; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; margin: 5px 0px 5px 5px; font-weight: bold; border: solid 1px #eecbff; padding: 2px 15px 2px 15px; font-size: 10px;\'>Rows</span><span style=\'background-color: none; color: #333333; padding: 0.5em 0.5em; position: inherit; margin: 5px 0px 5px -4px; font-weight: bold; border: solid 1px #eecbff; padding: 2px 15px 2px 15px; font-size: 10px;\'>13</span><span style=\'background-color: #BDE7B4; color: #333333; padding: 0.5em 0.5em; position: inherit; text-transform: uppercase; margin: 5px 0px 5px 3px; font-weight: bold; border: solid 1px #BDE7B4; padding: 2px 15px 2px 15px; font-size: 10px;\'>Columns</span><span style=\'background-color: none; color: #333333; padding: 0.5em 0.5em; position: inherit; margin: 5px 0px 5px -4px; font-weight: bold; border: solid 1px #BDE7B4; padding: 2px 15px 2px 15px; font-size: 10px;\'>8</span></div></div>'), preheader